In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('./data/smartwatches.csv')

In [5]:
df.head()

,Unnamed: 0,Brand,Current Price,Original Price,Discount Percentage,Rating,Number OF Ratings,Model Name,Dial Shape,Strap Color,Strap Material,Touchscreen,Battery Life (Days),Bluetooth,Display Size,Weight
0,0,noise,82990.0,89900.0,7.686318,4.0,65.0,Wrb-sw-colorfitpro4alpha-std-rgld_pnk,NaN,NaN,NaN,NaN,8.0,Yes,NaN,35 - 50 g
1,1,fire-boltt,3799.0,16999.0,77.651627,4.3,20788.0,BSW046,NaN,NaN,Silicon,Yes,3.5,Yes,1.8 inches,50 - 75 g
2,2,boat,1999.0,7990.0,74.981227,3.8,21724.0,Wave Call,NaN,NaN,Silicon,Yes,8.0,Yes,1.7 inches,35 - 50 g
3,3,fire-boltt,1799.0,19999.0,91.004550,4.3,13244.0,BSW053,NaN,NaN,Silicon,Yes,3.5,Yes,1.8 inches,75g +
4,4,noise,1599.0,4999.0,68.013603,4.1,13901.0,Wrb-sw-colorfitpulsegobuzz-std-blk_blk,NaN,NaN,Other,Yes,8.0,Yes,1.7 inches,35 - 50 g


In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           450 non-null    int64  
 1   Brand                450 non-null    object 
 2   Current Price        440 non-null    float64
 3   Original Price       377 non-null    float64
 4   Discount Percentage  377 non-null    float64
 5   Rating               444 non-null    float64
 6   Number OF Ratings    391 non-null    float64
 7   Model Name           415 non-null    object 
 8   Dial Shape           327 non-null    object 
 9   Strap Color          327 non-null    object 
 10  Strap Material       381 non-null    object 
 11  Touchscreen          413 non-null    object 
 12  Battery Life (Days)  418 non-null    float64
 13  Bluetooth            444 non-null    object 
 14  Display Size         420 non-null    object 
 15  Weight               264 non-null    obj

In [14]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [15]:
df.shape

(450, 15)

In [17]:
train_df = df.sample(frac=0.8, random_state=48)
train_df

,Brand,Current Price,Original Price,Discount Percentage,Rating,Number OF Ratings,Model Name,Dial Shape,Strap Color,Strap Material,Touchscreen,Battery Life (Days),Bluetooth,Display Size,Weight
422,fire-boltt,3449.0,9999.0,65.506551,3.8,83.0,ring,Square,Black,Rubber,Yes,22.0,Yes,0.1 inches,75g +
414,fire-boltt,3927.0,9999.0,60.726073,2.5,NaN,RING 3,Square,Gold,Silicon,Yes,22.0,Yes,1.8 inches,NaN
263,garmin,44990.0,50490.0,10.893246,4.0,6.0,"Instinct 2 Solar, Rugged Outdoor SmartWatch, M...",Circle,Blue,Silicon,No,17.5,Yes,0.9 inches,NaN
259,garmin,67490.0,NaN,NaN,4.7,NaN,"Forerunner 955 Solar charging upto 20 days, HR...",Circle,White,Silicon,No,8.0,Yes,1.3 inches,75g +
317,fossil,10998.0,21995.0,49.997727,3.9,5182.0,4th Gen Explorist HR,Circle,Black,Silicon,Yes,22.0,Yes,1.8 inches,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,fire-boltt,2499.0,9999.0,75.007501,4.1,2571.0,BSW072,NaN,NaN,NaN,Yes,8.0,Yes,2.0 inches,75g +
355,ambrane,1499.0,5299.0,71.711644,4.2,8574.0,"Wise Eon Pro1.85"" lucid display with BT calling",Square,Red,Silicon,Yes,22.0,Yes,1.9 inches,NaN
35,boat,1199.0,6990.0,82.846924,4.0,27771.0,Watch Flash,NaN,NaN,Silicon,Yes,8.0,Yes,1.3 inches,50 - 75 g
185,honor,13999.0,15999.0,12.500781,4.4,14613.0,Magic Watch 2 (46 mm),Circle,Brown,Leather,Yes,8.0,Yes,1.8 inches,20 - 35 g


In [19]:
test_df = df.drop(train_df.index)
test_df

,Brand,Current Price,Original Price,Discount Percentage,Rating,Number OF Ratings,Model Name,Dial Shape,Strap Color,Strap Material,Touchscreen,Battery Life (Days),Bluetooth,Display Size,Weight
0,noise,82990.0,89900.0,7.686318,4.0,65.0,Wrb-sw-colorfitpro4alpha-std-rgld_pnk,NaN,NaN,NaN,NaN,8.0,Yes,NaN,35 - 50 g
1,fire-boltt,3799.0,16999.0,77.651627,4.3,20788.0,BSW046,NaN,NaN,Silicon,Yes,3.5,Yes,1.8 inches,50 - 75 g
3,fire-boltt,1799.0,19999.0,91.004550,4.3,13244.0,BSW053,NaN,NaN,Silicon,Yes,3.5,Yes,1.8 inches,75g +
6,noise,1999.0,4999.0,60.012002,4.1,1018.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes,1.4 inches,35 - 50 g
12,noise,3799.0,7999.0,52.506563,3.8,545.0,Wrb-sw-colorfitpro4alpha-std-blk_blk,NaN,NaN,NaN,NaN,8.0,Yes,NaN,35 - 50 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,fire-boltt,4599.0,17999.0,74.448580,2.5,11.0,HULK BLUETOOTH CALLING,Square,Black,Silicon,Yes,22.0,Yes,1.8 inches,NaN
420,fire-boltt,2699.0,4999.0,46.009202,2.5,33.0,NINJA PRO,Square,Black,Silicon,Yes,22.0,Yes,1.3 inches,NaN
425,fire-boltt,3999.0,9999.0,60.006001,3.7,NaN,bsw360,Circle,Blue,Rubber,Yes,22.0,Yes,0.1 inches,NaN
432,fire-boltt,3699.0,11999.0,69.172431,2.5,NaN,starcross,Square,Black,Silicon,Yes,8.0,Yes,1.8 inches,NaN


In [20]:
print(test_df.shape, train_df.shape)

(90, 15) (360, 15)
